## "Laplacian Pyramid"

This notebook is ment to demonstrate my objections concerning the implementaiton of the laplacian pyramid in DRLN.
Please remember their proposed desing of the attention mechanism in their paper. They aim to produce attention differenlty at each pyramid by using different downsampling layers.

![Attention](figs/LapAtt.png)
In the following cell is the original code of their implementation of the attention layer (CA). As you can see they used three different streames (c1,c2,c3) which are ment to be their "pyramids". However, the only parameters they change is the dilation and padding parameter of a 2D convolution as you can see in the BaiscBlock class they use.

We need to install create a new conda environment becaus we need a newer version of python due to some bug.

In [1]:
%%bash
conda env create -f torch.yml # may take a while
python -m ipykernel install --user --name=trytorch # activating environment for jupyter notebook
# you may need to close and open the notebook again 

Installed kernelspec trytorch in /home/charlie/.local/share/jupyter/kernels/trytorch



CondaValueError: prefix already exists: /home/charlie/anaconda3/envs/trytorch



In [2]:
import torch
import torch.nn as nn
import numpy as np

In [3]:
class CALayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(CALayer, self).__init__()

        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.c1 = ops.BasicBlock(channel , channel // reduction, 3, 1, 3, 3)
        self.c2 = ops.BasicBlock(channel , channel // reduction, 3, 1, 5, 5)
        self.c3 = ops.BasicBlock(channel , channel // reduction, 3, 1, 7, 7)
        self.c4 = ops.BasicBlockSig((channel // reduction)*3, channel , 3, 1, 1)

    def forward(self, x):
        print(x.shape, "input")
        y = self.avg_pool(x)
        print(y.shape, "after pooling")
        c1 = self.c1(y)
        c2 = self.c2(y)
        c3 = self.c3(y)
        print(c1.shape, c2.shape, c3.shape, "shapes")
        c_out = torch.cat([c1, c2, c3], dim=1)
        y = self.c4(c_out)
        return x * y

class BasicBlock(nn.Module):
    def __init__(self,
                 in_channels, out_channels,
                 ksize=3, stride=1, pad=1, dilation=1):
        super(BasicBlock, self).__init__()

        self.body = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, ksize, stride, pad, dilation),
            nn.ReLU(inplace=True)
        )

        init_weights(self.modules)
        
    def forward(self, x):
        out = self.body(x)
        return out


But with a input of size (batch_size, channels,
1, 1), padding it with the exact amount of zeros needed to fit the
increased dilation will render all weights of the convolution layers
except the center ones useless. Thus, it seems to me that they do the
same computational process in each pyramid which is also the same as the
simple CA used by RCAN but with introducing unnecessary weights (i.e.
weights that always only get the input zero from the padding) by
changing padding and dilation. I tried it myself, and came to the
conclusion that one could produce the same result with just using 1x1
kernels without padding and dilation, thus getting rid of quite a few
parameters.
I tried to produce a simple visualization of my remark (see picture below) and wrote the following code to support my claim.
![](figs/DRLN_cnn_dilation_and_padding.png)


In [4]:
# the different layers in the Attention Model
channels_in = 64
reduction = 16
channels_out = channels_in // reduction
dilation3 = nn.Conv2d(channels_in, channels_out, 3,1,3,3, bias=False)
dilation5 = nn.Conv2d(channels_in, channels_out, 3,1,5,5, bias=False)
dilation7 = nn.Conv2d(channels_in, channels_out, 3,1,7,7, bias=False)

In [5]:
# setting the weights to 1
conv_all_one = []
for conv in [dilation3, dilation5, dilation7]:
    #conv.weight = nn.Parameter(torch.ones(conv.weight.shape))
    print(f"{conv} weights: {conv.weight} ")
    conv_all_one.append(conv)

Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), dilation=(3, 3), bias=False) weights: Parameter containing:
tensor([[[[-0.0296, -0.0160, -0.0261],
          [ 0.0074, -0.0366, -0.0329],
          [-0.0074, -0.0326,  0.0283]],

         [[-0.0049,  0.0338, -0.0197],
          [ 0.0191,  0.0405, -0.0276],
          [ 0.0042, -0.0380,  0.0201]],

         [[ 0.0308, -0.0239, -0.0062],
          [ 0.0298, -0.0039, -0.0342],
          [-0.0376, -0.0120, -0.0009]],

         ...,

         [[ 0.0155,  0.0216,  0.0206],
          [ 0.0246,  0.0229, -0.0130],
          [ 0.0151,  0.0111, -0.0261]],

         [[-0.0092,  0.0286,  0.0212],
          [ 0.0213,  0.0349, -0.0008],
          [ 0.0405, -0.0013,  0.0061]],

         [[ 0.0008,  0.0283,  0.0324],
          [ 0.0035,  0.0311, -0.0252],
          [-0.0416, -0.0292, -0.0342]]],


        [[[ 0.0067,  0.0026,  0.0203],
          [ 0.0078,  0.0066,  0.0337],
          [-0.0021, -0.0178,  0.0308]],

         [[-0.0322, -0.03

In [6]:
# defining a dummy input of shape (1 (batch_size), channels in, 1, 1)
arr = np.arange(channels_in).reshape(1,channels_in, 1, 1)
dummy = torch.tensor(arr, dtype=torch.float)
print(f"dummy input has chape {dummy.shape}")
print(dummy)

dummy input has chape torch.Size([1, 64, 1, 1])
tensor([[[[ 0.]],

         [[ 1.]],

         [[ 2.]],

         [[ 3.]],

         [[ 4.]],

         [[ 5.]],

         [[ 6.]],

         [[ 7.]],

         [[ 8.]],

         [[ 9.]],

         [[10.]],

         [[11.]],

         [[12.]],

         [[13.]],

         [[14.]],

         [[15.]],

         [[16.]],

         [[17.]],

         [[18.]],

         [[19.]],

         [[20.]],

         [[21.]],

         [[22.]],

         [[23.]],

         [[24.]],

         [[25.]],

         [[26.]],

         [[27.]],

         [[28.]],

         [[29.]],

         [[30.]],

         [[31.]],

         [[32.]],

         [[33.]],

         [[34.]],

         [[35.]],

         [[36.]],

         [[37.]],

         [[38.]],

         [[39.]],

         [[40.]],

         [[41.]],

         [[42.]],

         [[43.]],

         [[44.]],

         [[45.]],

         [[46.]],

         [[47.]],

         [[48.]],

         [[49.]],

  

In [7]:
# now we do the forward pass with our 3 cnn layers
def compute_result(dummy, dilations):
    results = []
    for dil in dilations:
        results.append(dil(dummy))
    return results

In [8]:
results_all_one = compute_result(dummy, conv_all_one)
print(results_all_one)

[tensor([[[[-5.9458]],

         [[ 1.9389]],

         [[-1.7697]],

         [[ 3.9780]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[ 2.8810]],

         [[-1.8030]],

         [[12.4181]],

         [[ 1.8972]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[ 4.7673]],

         [[-0.8068]],

         [[10.0455]],

         [[ 9.6450]]]], grad_fn=<ThnnConvDilated2DBackward>)]


If we now sett all weights to 0.1 the output will just be a tensor of shape (1,4,1,1) with all values being 0.1 * 2016.

In [9]:
conv_all_point_one = []
for conv in [dilation3, dilation5, dilation7]:
    conv.weight = nn.Parameter(torch.ones(conv.weight.shape))
    conv.weight[:] = 0.1
    print(f"{conv} weights: {conv.weight} ")
    conv_all_point_one.append(conv)

Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), dilation=(3, 3), bias=False) weights: Parameter containing:
tensor([[[[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         ...,

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]]],


        [[[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.100

In [10]:
results_all_point_one = compute_result(dummy, conv_all_point_one)
print(results_all_point_one)

[tensor([[[[201.6000]],

         [[201.6000]],

         [[201.6000]],

         [[201.6000]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[201.6000]],

         [[201.6000]],

         [[201.6000]],

         [[201.6000]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[201.6000]],

         [[201.6000]],

         [[201.6000]],

         [[201.6000]]]], grad_fn=<ThnnConvDilated2DBackward>)]


In [11]:
# now we set all weights to 0.1 except the one in the iddle which will be 1
conv_one_one = []
for conv in [dilation3, dilation5, dilation7]:
    conv.weight[:] = 0.1
    # the middle weight determines the output
    # 1 -> 2016
    conv.weight[:,:,1,1] = 1
    # everything else will not matter
    print(f"{conv} weights: {conv.weight} ")
    conv_one_one.append(conv)

Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), dilation=(3, 3), bias=False) weights: Parameter containing:
tensor([[[[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         ...,

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.1000, 0.1000, 0.1000]]],


        [[[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.1000, 0.1000, 0.1000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 1.0000, 0.1000],
          [0.100

In [14]:
results_one_one = compute_result(dummy, conv_one_one)
print(results_one_one)

[tensor([[[[2016.]],

         [[2016.]],

         [[2016.]],

         [[2016.]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[2016.]],

         [[2016.]],

         [[2016.]],

         [[2016.]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[2016.]],

         [[2016.]],

         [[2016.]],

         [[2016.]]]], grad_fn=<ThnnConvDilated2DBackward>)]


In fact we can choos all the weights we want for the non-center weights of our kernel.

In [15]:
# now we set all weights to 0.1 except the one in the iddle which will be 1
conv_one_one_rest_random = []
for conv in [dilation3, dilation5, dilation7]:
    conv.weight[:] = 0.1
    # the middle weight determines the output
    # 1 -> 2016
    conv.weight[:,:,1,1] = 1
    # everything else will not matter
    conv.weight[:,0,2,2] = 10000000000000000
    conv.weight[2,1,0,0] = 0
    conv.weight[1,1,2,2] = np.pi
    print(f"{conv} weights: {conv.weight} ")
    conv_one_one_rest_random.append(conv)

Conv2d(64, 4, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), dilation=(3, 3), bias=False) weights: Parameter containing:
tensor([[[[1.0000e-01, 1.0000e-01, 1.0000e-01],
          [1.0000e-01, 1.0000e+00, 1.0000e-01],
          [1.0000e-01, 1.0000e-01, 1.0000e+16]],

         [[1.0000e-01, 1.0000e-01, 1.0000e-01],
          [1.0000e-01, 1.0000e+00, 1.0000e-01],
          [1.0000e-01, 1.0000e-01, 1.0000e-01]],

         [[1.0000e-01, 1.0000e-01, 1.0000e-01],
          [1.0000e-01, 1.0000e+00, 1.0000e-01],
          [1.0000e-01, 1.0000e-01, 1.0000e-01]],

         ...,

         [[1.0000e-01, 1.0000e-01, 1.0000e-01],
          [1.0000e-01, 1.0000e+00, 1.0000e-01],
          [1.0000e-01, 1.0000e-01, 1.0000e-01]],

         [[1.0000e-01, 1.0000e-01, 1.0000e-01],
          [1.0000e-01, 1.0000e+00, 1.0000e-01],
          [1.0000e-01, 1.0000e-01, 1.0000e-01]],

         [[1.0000e-01, 1.0000e-01, 1.0000e-01],
          [1.0000e-01, 1.0000e+00, 1.0000e-01],
          [1.0000e-01, 1.0000e-01,

In [16]:
results_one_one_rest_random = compute_result(dummy, conv_one_one_rest_random)
print(results_one_one_rest_random)

[tensor([[[[2016.]],

         [[2016.]],

         [[2016.]],

         [[2016.]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[2016.]],

         [[2016.]],

         [[2016.]],

         [[2016.]]]], grad_fn=<ThnnConvDilated2DBackward>), tensor([[[[2016.]],

         [[2016.]],

         [[2016.]],

         [[2016.]]]], grad_fn=<ThnnConvDilated2DBackward>)]


We see that each output has the same dimensions and the same values.
Morover, only changing the dilation and the padding value will render all weights of our dilation useless except the one in the middle which will be multiplied with our 1x1 input.
    We can demonstrate that by setting all weights to 0 except the one in the middle. Thus we could achive the same when using convolutions with 1x1 kernels instead.

In [17]:
channels_in = 64
reduction = 16
channels_out = channels_in // reduction
simple_layers = []
for i in range(3):
    simple_layers.append(nn.Conv2d(channels_in, channels_out, 1, bias=False))
    simple_layers[i].weight = nn.Parameter(torch.ones(simple_layers[i].weight.shape))
    simple_layers[i].weight[:] = 0.1 
    print(f"{simple_layers[i]} weights: {simple_layers[i].weight.shape} ")
    

Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False) weights: torch.Size([4, 64, 1, 1]) 
Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False) weights: torch.Size([4, 64, 1, 1]) 
Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False) weights: torch.Size([4, 64, 1, 1]) 


In [ ]:
results = compute_result(dummy, simple_layers)
print(results)

